In [1]:
import pandas as pd
import polars as pl
import ManipulateData
import DataLoader_Polars
from rapidfuzz import fuzz
import jellyfish
import re
import matplotlib.pyplot as plt

In [2]:
data_loader = DataLoader_Polars.DataLoaderPolars("../DataSets_CSVs/DataSets_Parquets/")
df_concesiones = data_loader.load_concesiones()
manipulator = ManipulateData.ManipulateData(df_concesiones)

In [3]:
titulares_unicos = df_concesiones.sort('titular').select('titular').unique()

In [4]:
n_nombres_titulares_antes_estandarizar = len(df_concesiones.select('titular').unique())
n_nombres_titulares_antes_estandarizar

544820

In [5]:
#df_concesiones = ManipulateData.ManipulateData(df_concesiones).estandarizar_titular('titular',
#                                                                                    sin_espacios=False,
#                                                                                    sin_stopword=True,
#                                                                                    sin_terminos_mercantiles=True,
#                                                                                    quitar_tokens_1_letra=True)

In [6]:
n_nombres_titulares_despues_estandarizar = len(df_concesiones.select('titular').unique())
n_nombres_titulares_despues_estandarizar

544820

In [7]:
print(n_nombres_titulares_antes_estandarizar - n_nombres_titulares_despues_estandarizar)

0


In [8]:
# Seleccionar columnas en el orden deseado

df_concesiones = df_concesiones.select([
    'titulo', 'titular', 'ultimo_registro',
    *[col for col in df_concesiones.columns if col not in ['titulo', 'titular', 'ultimo_registro']]
])


In [9]:
# Ordenar el DataFrame
df_concesiones = df_concesiones.sort([
    pl.col('titulo').count().over('titulo'),
    'titular', 
    'ultimo_registro'
], descending=[True, False, False])

In [10]:
# Obtener información de títulos y titulares
titulos_info = df_concesiones.group_by('titulo').agg([
    pl.col('titular').n_unique().alias('N_titulares')
]).sort('N_titulares', descending=True)



In [11]:
volumenes_totales_info = df_concesiones.group_by('titulo').agg([
    pl.col('volumen_de_extraccion_anual_de_aguas_nacionales_que_ampara_el_titulo_en_m3').n_unique().alias('cambios_volumen'),
    pl.col('volumen_de_extraccion_anual_de_aprovechamientos_subterraneos_en_m3').n_unique().alias('cambios_volumen_subterraneo'),
    pl.col('volumen_de_extraccion_anual_de_aprovechamientos_superficiales_en_m3').n_unique().alias('cambios_volumen_superficial')
])

In [12]:
# Recrear df_anomalias con los datos
df_anomalias = pl.DataFrame({
    'titulo': titulos_info['titulo'].to_list(),
    'N_titulares': titulos_info['N_titulares'].to_list(),
    'Score': [None] * len(titulos_info),
    'Diferente_titular': [None] * len(titulos_info),
    'Anomalia_V_Total': volumenes_totales_info['cambios_volumen'].to_list(),
    'Anomalia_V_Subterraneo': volumenes_totales_info['cambios_volumen_subterraneo'].to_list(),
    'Anomalia_V_Superficial': volumenes_totales_info['cambios_volumen_superficial'].to_list()
})

In [13]:
ALNUM = re.compile(r'[A-Z0-9]+')

In [22]:
df_concesiones

titulo,titular,ultimo_registro,uso_que_ampara_el_titulo,volumen_de_extraccion_anual_de_aguas_nacionales_que_ampara_el_titulo_en_m3,aprovechamientos_superficiales_que_ampara_el_titulo,volumen_de_extraccion_anual_de_aprovechamientos_superficiales_en_m3,aprovechamientos_subterraneos_que_ampara_el_titulo,volumen_de_extraccion_anual_de_aprovechamientos_subterraneos_en_m3,puntos_de_descarga_que_ampara_el_titulo,volumen_de_descarga_diario_en_m3,zonas_federales_que_ampara_el_titulo,superficie_en_m2,fecha_de_registro,fecha_de_inicio_de_vigencia,fecha_de_termino_de_vigencia,autoridad_que_emite_el_acto
str,str,str,str,f64,i64,f64,i64,f64,i64,f64,i64,f64,str,str,str,str
"""11CHS128762/23GPGR99""","""DAGOBERTO ARMANDO OCHOA DIAZ""","""abril 2019""","""PECUARIO""",2234.9,0,0.0,1,2234.9,0,0.0,0,0.0,"""2000-01-13""",null,null,null
"""11CHS128762/23GPGR99""","""DAGOBERTO ARMANDO OCHOA DIAZ""","""agosto 2024""","""PECUARIO""",2234.9,0,0.0,1,2234.9,0,0.0,0,0.0,"""2000-01-13""",null,null,"""DIRECCIÓN GENERAL DEL ORGANISM…"
"""11CHS128762/23GPGR99""","""DAGOBERTO ARMANDO OCHOA DIAZ""","""diciembre 2020""","""PECUARIO""",2234.9,0,0.0,1,2234.9,0,0.0,0,0.0,"""2000-01-13""",null,null,"""DIRECCION GENERAL DEL ORGANISM…"
"""11CHS128762/23GPGR99""","""DAGOBERTO ARMANDO OCHOA DIAZ""","""diciembre 2021""","""PECUARIO""",2234.9,0,0.0,1,2234.9,0,0.0,0,0.0,"""13-01-20""",null,null,"""DIRECCIÓN GENERAL DEL ORGANISM…"
"""11CHS128762/23GPGR99""","""DAGOBERTO ARMANDO OCHOA DIAZ""","""enero 2020""","""PECUARIO""",2234.9,0,0.0,1,2234.9,0,0.0,0,0.0,"""2000-01-13""",null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""MOR155124""","""YOLANDA SEGURA ROSAS""","""marzo 2025""","""AGRÍCOLA""",0.0,0,0.0,0,0.0,0,0.0,1,212.44,"""2015-02-04""",null,null,"""ORGANISMO DE CUENCA BALSAS"""
"""GUA109671""","""YSMAEL ESPINOSA RICO""","""marzo 2025""","""AGRÍCOLA""",36000.0,0,0.0,1,36000.0,0,0.0,0,0.0,"""1998-12-15""",null,null,"""DIRECCION LOCAL GUANAJUATO"""
"""GUA120376""","""ZAAC S.A. DE C.V.""","""marzo 2025""","""AGRÍCOLA""",240000.0,0,0.0,1,240000.0,0,0.0,0,0.0,"""2006-11-24""",null,null,"""DIRECCION LOCAL GUANAJUATO"""


In [ ]:
def score_y_clase(a:str, b:str):
    #metricas
    ts = fuzz.token_set_ratio(a, b)
    jw = jellyfish.jaro_winkler_similarity(a, b) * 100
    score = (ts + jw) / 2
    
    #reglas
    if ts >= 96 or (ts >= 93 and jw >= 93):
        clase = 1 #iguales
    elif ts < 90 or (ts < 85 and jw < 88):
        clase = 0 #diferentes
    else:
        clase = 2 
    return score, clase

In [15]:
# Títulos con exactamente 2 titulares únicos
tit2 = (
    df_concesiones
    .select(['titulo','titular'])
    .unique()
    .group_by('titulo')
    .agg(pl.col('titular').unique().alias('titulares'))
    .with_columns(pl.col('titulares').list.len().alias('N_titulares'))
    .filter(pl.col('N_titulares') == 2)
)

# Extrae pares (t1, t2)
tit2_pairs = tit2.with_columns([
    pl.col('titulares').list.get(0).alias('titular1'),
    pl.col('titulares').list.get(1).alias('titular2')
]).select(['titulo','titular1','titular2'])

In [16]:
tit2_scored = tit2_pairs.with_columns([
    pl.struct(['titular1','titular2']).map_elements(
        lambda s: score_y_clase(s['titular1'], s['titular2'])[0]
    ).alias('Score'),
    pl.struct(['titular1','titular2']).map_elements(
        lambda s: score_y_clase(s['titular1'], s['titular2'])[1]
    ).alias('Diferente_titular')
])

/tmp/ipykernel_36234/2404338928.py:1: MapWithoutReturnDtypeWarning: 'return_dtype' of function python_udf must be set

A later expression might fail because the output type is not known. Set return_dtype=pl.self_dtype() if the type is unchanged, or set the proper output data type.
  tit2_scored = tit2_pairs.with_columns([


In [21]:
df_concesiones.select('ultimo_registro')

ultimo_registro
str
"""abril 2019"""
"""agosto 2024"""
"""diciembre 2020"""
"""diciembre 2021"""
"""enero 2020"""
…
"""marzo 2025"""
"""marzo 2025"""
"""marzo 2025"""


In [17]:
df_anomalias = (
    df_anomalias.drop(['Score','Diferente_titular'])
    .join(tit2_scored.select(['titulo','Score','Diferente_titular']), on='titulo', how='left')
    .select([
        'titulo',
        'N_titulares',
        'Score',
        'Diferente_titular', 
        'Anomalia_V_Total',
        'Anomalia_V_Subterraneo',
        'Anomalia_V_Superficial'
    ])
)

In [18]:
df_anomalias

titulo,N_titulares,Score,Diferente_titular,Anomalia_V_Total,Anomalia_V_Subterraneo,Anomalia_V_Superficial
str,i64,f64,i64,i64,i64,i64
"""810978""",7,null,null,1,1,1
"""04MOR104010/18AMDA15""",6,null,null,1,1,1
"""824314""",6,null,null,1,1,1
"""815222""",6,null,null,1,1,1
"""GUA106928""",6,null,null,1,1,1
…,…,…,…,…,…,…
"""10VER132526/28APGR04""",1,null,null,1,1,1
"""08GUA102539/12AMGE06""",1,null,null,1,1,1
"""08GUA115736/12AMDL14""",1,null,null,1,1,1


In [19]:

# fechas = df_concesiones.select('ultimo_registro').unique().sort('ultimo_registro')
# titulos_list = df_concesiones.group_by('titulo').agg(pl.count()).sort('count', descending=True)
# nombres_unicos_en_titulares = df_concesiones.sort('titular').select('titular').unique()
# Distribución de títulos por frecuencia
#dist_titulos_list = titulos_list.group_by('count').agg(pl.count().alias('frequency')).sort('count',descending=True)

In [20]:
'''
plt.bar(dist_titulos_list['count'].to_list(), dist_titulos_list['frequency'].to_list())
plt.yscale('log')
plt.show()
'''

"\nplt.bar(dist_titulos_list['count'].to_list(), dist_titulos_list['frequency'].to_list())\nplt.yscale('log')\nplt.show()\n"